In [3]:
import argparse
import os
import numpy as np
import torch
import cv2, glob
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from modeling.sync_batchnorm.replicate import patch_replication_callback
from modeling.deeplab import *

from dataloaders.utils import decode_segmap
MI = np.load('meanimage2.npy')

numClass={'pascal':2,
'coco':21,
'cityscapes':19}
if not os.path.isdir('result_task2/'):
    os.mkdir('result_task2/')
    os.mkdir('result_task2/color')

# parser = argparse.ArgumentParser(description="PyTorch DeeplabV3Plus Training")

# parser.add_argument('--backbone', type=str, default='resnet',choices=['resnet', 'xception', 'drn', 'mobilenet'],help='backbone name (default: resnet)')
# parser.add_argument('--out-stride', type=int, default=16,help='network output stride (default: 8)')
# parser.add_argument('--dataset', type=str, default='cityscapes', choices=['pascal', 'coco', 'cityscapes'],help='dataset name (default: pascal)')
# parser.add_argument('--sync-bn', type=bool, default=None,help='whether to use sync bn (default: auto)')
# parser.add_argument('--freeze-bn', type=bool, default=False,help='whether to freeze bn parameters (default: False)')
# parser.add_argument('--weightPath', type=str, default=None,help='put the path to resuming file if needed')
# parser.add_argument('--imgPath', type=str, default=None,help='put the path to resuming file if needed')
# parser.add_argument('--outPath', type=str, default=None,help='put the path to resuming file if needed')
# parser.add_argument('--no-cuda', action='store_true', default=False, help='disables CUDA training')
# args = parser.parse_args()
cuda = torch.cuda.is_available()
nclass = numClass['pascal']
model = DeepLab(num_classes=nclass, backbone='resnet', output_stride=16, sync_bn=None, freeze_bn=False)
weight_dict=torch.load('run/pascal/task2/model_best.pth.tar', map_location='cpu')
# if cuda:
#     model = torch.nn.DataParallel(model, device_ids=[0])
#     patch_replication_callback(model)
#     model = model.cuda()
    
# model.module.load_state_dict(weight_dict['state_dict'])
model.load_state_dict(weight_dict['state_dict'])

model.eval()


filenames = glob.glob('Validation-400-images/*.jpg')
kernel = np.ones((5,5),np.uint8)
kernel2 = np.ones((7,7),np.uint8)
for imgPath in filenames:
    fn = os.path.basename(imgPath)[:-4]
    outPath = 'result_task2/color/' + fn +'.png'

    image = cv2.imread(imgPath)
    oriDim = image.shape
    image = cv2.resize(image, dsize=(513,513)) - MI
    image = image.astype(np.float32) / 255.
    means = [0.485, 0.456, 0.406]
    stds = [0.229, 0.224, 0.225]
    
    
    image = image[:, :, ::-1]
    for i in range(3):
        image[:, :, i] = image[:, :, i] - means[i]
        image[:, :, i] = image[:, :, i] / stds[i]

    image = torch.from_numpy(image.transpose((2, 0, 1)).astype(np.float32)).float().unsqueeze(0)

#     image = image.cuda()
    with torch.no_grad():
        output = model(image)
        output = output.data.cpu().numpy()
        prediction = np.argmax(output, axis=1)[0]
        ps=[]

        mask = np.zeros((prediction.shape[0], prediction.shape[1]), np.uint8) +255
        mask[prediction == 1] = 0
        mask = cv2.morphologyEx(255-mask, cv2.MORPH_OPEN, kernel)
        mask = 255-cv2.morphologyEx(mask, cv2.MORPH_DILATE, kernel2)

        mask = cv2.resize(mask,dsize=(oriDim[1],oriDim[0]), interpolation=cv2.INTER_NEAREST)
        ps.append(np.mean(prediction == 1))
        cv2.imwrite('result_task2/' + fn+'.png', mask)

        segmap = decode_segmap(prediction, dataset='pascal')
        segmap = (segmap*255).astype(np.uint8)
        segmap = cv2.resize(segmap,dsize=(oriDim[1],oriDim[0]))
        segmap = segmap[:, :, ::-1]
        cv2.imwrite(outPath, segmap)
    print('Done inference '+fn,'percentage:', ps)
exit(1)

Done inference V0366 percentage: [0.0]
Done inference V0213 percentage: [0.03126888045324487]
Done inference V0333 percentage: [0.0]
Done inference V0108 percentage: [0.025246134613119326]
Done inference V0264 percentage: [0.040118706990564995]
Done inference V0323 percentage: [0.02367300099935783]
Done inference V0347 percentage: [0.030197325672856604]
Done inference V0064 percentage: [0.03788820111791282]
Done inference V0389 percentage: [6.839711364180432e-05]
Done inference V0381 percentage: [0.03200984918436442]
Done inference V0306 percentage: [0.032435431224802316]
Done inference V0307 percentage: [0.0222176624146461]
Done inference V0188 percentage: [0.04019470378350034]
Done inference V0014 percentage: [0.035061120420718246]
Done inference V0296 percentage: [0.0018315227097416488]
Done inference V0282 percentage: [0.028696389012383677]
Done inference V0325 percentage: [0.01948937754826746]
Done inference V0057 percentage: [0.028992776504831497]
Done inference V0149 percentage:

Done inference V0068 percentage: [0.02341081206373091]
Done inference V0193 percentage: [0.02761343471305511]
Done inference V0090 percentage: [0.029980734812990892]
Done inference V0132 percentage: [0.01822783078554085]
Done inference V0335 percentage: [0.022016270913367456]
Done inference V0150 percentage: [3.7998396467669062e-06]
Done inference V0026 percentage: [0.023042227617994522]
Done inference V0036 percentage: [0.04396794455273988]
Done inference V0387 percentage: [0.02737784465495556]
Done inference V0151 percentage: [0.035239712884116293]
Done inference V0005 percentage: [0.03319919899380246]
Done inference V0037 percentage: [0.024132781596616623]
Done inference V0315 percentage: [0.024474767164825644]
Done inference V0257 percentage: [0.03137147612370758]
Done inference V0221 percentage: [0.02842280055781646]
Done inference V0378 percentage: [0.02383259426452204]
Done inference V0158 percentage: [0.03404656323503148]
Done inference V0045 percentage: [0.014169602042793794]


Done inference V0199 percentage: [0.03249622865915058]
Done inference V0143 percentage: [0.026925663736990298]
Done inference V0106 percentage: [0.02505614263078098]
Done inference V0292 percentage: [0.0]
Done inference V0322 percentage: [0.02597950366494534]
Done inference V0183 percentage: [0.05278737237288587]
Done inference V0052 percentage: [0.0005205780316070662]
Done inference V0050 percentage: [0.03720043014184801]
Done inference V0060 percentage: [0.01873320945856085]
Done inference V0137 percentage: [0.004662403246582994]
Done inference V0393 percentage: [0.024695157864338124]
Done inference V0094 percentage: [0.027704630864577515]
Done inference V0383 percentage: [0.01665849701142612]
Done inference V0362 percentage: [0.0047270005205780315]
Done inference V0070 percentage: [0.029817341708179915]
Done inference V0304 percentage: [0.02970334651877691]
Done inference V0191 percentage: [0.021898475884317682]
Done inference V0153 percentage: [0.018691411222446414]
Done inference 